# Encoding analysis

In [1]:
%matplotlib inline
projecttitle = 'Analogy'
import sys, os
if sys.platform == 'darwin':
    
    sys.path.append(os.path.join("/Users", "njchiang", "GitHub", "task-fmri-utils"))
else:
    sys.path.append(os.path.join("D:\\", "GitHub", "task-fmri-utils"))

In [2]:
from fmri_core import analysis as pa
from fmri_core import utils as pu

In [3]:
os.chdir("D:\\CloudStation\\Grad\\Research\\montilab-ucla")

In [4]:
projectSettings = pu.loadConfig(os.path.join('analogy', 'config', 'project.json'))
analysisSettings = pu.loadConfig(os.path.join('analogy', 'config', 'analyses.json'))

Loading JSON config from analogy\config\project.json
Loading JSON config from analogy\config\analyses.json


In [4]:
# paths = projectSettings['filepaths']['osxPaths']
paths = projectSettings['filepaths']['winPaths']

Searchlight Analysis

In [5]:
analysisSettings['encoding']

{'param': 0,
 'postprob_template': '{}.mat',
 'rstpostprob79': 'rstpostprob',
 'rstpostprob9': 'rstpostprob_sm'}

In [6]:
# for each subject
# mask = lIFGoperc_bin-mask
def runsubject(sub, mask):
    # set image data
    import numpy as np
    imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas', pa.formatBIDSName(sub, 'task-analogy', 'betas-pymvpa.nii.gz'))
    mask = pa.loadImg(paths['root'], 
                      'derivatives', sub, 'masks', 
                      mask + '.nii.gz')
    labels = pa.loadLabels(paths['root'], 
                           'derivatives', sub, 'betas', 
                           pa.formatBIDSName(sub, 'task-analogy', 'events-pymvpa.tsv'), 
                           sep='\t', index_col=0)
    # load image
    maskedImg = pa.maskImg(imgFile, mask)
    # clean out timepoints of interest
    conditionSelector = labels['ab'] == 1

    # preprocessing
    from sklearn.preprocessing import StandardScaler
    op = StandardScaler()
    scaledData = pa.opByLabel(maskedImg, labels['chunks'], op)
    # analysis
    from sklearn.svm import SVC
    clf = SVC()
    from sklearn.model_selection import LeaveOneGroupOut
    cv = LeaveOneGroupOut()
    result = pa.searchlight(pa.unmaskImg(scaledData[conditionSelector], mask), 
                            labels['abmainrel'][conditionSelector], m=mask, cv=cv, 
                            groups=labels['chunks'][conditionSelector],
                            **analysisSettings['searchlight'])
    
    return result

In [7]:
test = runsubject('sub-01', 'lIFGoperc-bin_mask')

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\lIFGoperc-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_betas-pymvpa.nii.gz
starting searchlight... 
searchlight params: {'process_mask_img': None, 'radius': 5, 'n_jobs': -1, 'verbose': 0}


In [8]:
test.scores_.shape

(104, 104, 60)

In [24]:
import numpy as np
frames=30
groups = np.arange(frames) > frames/2 
groups = groups.astype(np.int32) + 1
groups

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2])

Can use PredefinedSplit